범주형 현수 --> OneHotEncoder --> 이항변수화 (0 or 1)  
성별 (남:0 , 여:1) --> 남 :01 , 여 : 10  
나이 (20대:0, 30대:1, 40대:2) --> 


In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
# 범주형 변수를 이항변수로 만들기 위해서


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [3]:
# 학생 ID ,성별 ,연령대, 성적

# ID : sn_1 ~sn_5
# Gender : 'm' , 'f'
# Age : 20대(0) , -- , 40대(2)
# Grade : A , -- , F

data = np.array([[0,0,0],
         [0,1,1],
         [0,2,2], # 남자,40대,C학점
         [1,0,3],
         [1,1,4]])
data

array([[0, 0, 0],
       [0, 1, 1],
       [0, 2, 2],
       [1, 0, 3],
       [1, 1, 4]])

In [ ]:
# 1,0,3 ==> 01 100 00010
# 1,1,4 ==> 01 010 00001

In [4]:
ohe=OneHotEncoder()
ohe.fit(data) # fitting-적합시킨다

OneHotEncoder()

In [6]:
ohe.transform(data).toarray()

array([[1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 1.]])

__Binerizer : 연속형 변수 -> 이항변수화 or digitize() , np.where()__  
__OneHotEncoder : 범주형 데이터 -> 이항변수화__

In [8]:
np.random.seed(11)

In [27]:
df=pd.DataFrame({
    'C1':np.random.randn(20),
    'C2':list('a'*10+'b'*10)
})
df

,C1,C2
0,1.095630,a
1,-0.309577,a
2,0.725752,a
3,1.549072,a
4,0.630080,a
5,0.073493,a
6,0.732271,a
7,-0.642575,a
8,-0.178093,a
9,-0.573955,a


In [10]:
print(df['C1'].max(),df['C1'].min())

1.7494547413051793 -2.653318559261479


In [13]:
Bins=np.linspace(df['C1'].min(),df['C1'].max(),10)
Bins

array([-2.65331856, -2.16412153, -1.67492449, -1.18572746, -0.69653043,
       -0.20733339,  0.28186364,  0.77106067,  1.26025771,  1.74945474])

In [15]:
df['C1_bin']=np.digitize(df['C1'],bins=Bins)
df.head(2)

,C1,C2,C1_bin
0,1.749455,a,10
1,-0.286073,a,5


In [16]:
df.groupby(['C1_bin'])['C1'].mean()

C1_bin
1    -2.653319
3    -1.305549
4    -1.023773
5    -0.464346
6    -0.019680
7     0.544833
9     1.574634
10    1.749455
Name: C1, dtype: float64

In [17]:
df.groupby(['C1_bin'])['C2'].value_counts()

C1_bin  C2
1       a     1
3       b     1
4       b     2
        a     1
5       a     4
        b     2
6       a     1
        b     1
7       b     3
        a     2
9       b     1
10      a     1
Name: C2, dtype: int64

In [19]:
# df의 C1_bin 의 값이 2와 같은 데이터만 출력
df[df['C1_bin']==5]

,C1,C2,C1_bin
1,-0.286073,a,5
2,-0.484565,a,5
5,-0.319631,a,5
6,-0.536629,a,5
11,-0.475733,b,5
19,-0.683447,b,5


__get_dummies()__

In [21]:
fruit=pd.DataFrame({
    'name':['apple','banana','cherry','durian',np.nan],
    'color':['red','yellow','red','green',np.nan]
})
fruit

,name,color
0,apple,red
1,banana,yellow
2,cherry,red
3,durian,green
4,NaN,NaN


In [22]:
pd.get_dummies(fruit['name'])  # nan 은 포함이 안된다.

,apple,banana,cherry,durian
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,0,0,0,0


In [24]:
pd.get_dummies(fruit)  # nan 도 설정이 되긴함?

,name_apple,name_banana,name_cherry,name_durian,color_green,color_red,color_yellow
0,1,0,0,0,0,1,0
1,0,1,0,0,0,0,1
2,0,0,1,0,0,1,0
3,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0


In [25]:
pd.get_dummies(fruit,columns=['name'],dummy_na=True)  # 거의 없긴하지만 nan도 포함해서 인코딩하는 방법

,color,name_apple,name_banana,name_cherry,name_durian,name_nan
0,red,1,0,0,0,0
1,yellow,0,1,0,0,0
2,red,0,0,1,0,0
3,green,0,0,0,1,0
4,NaN,0,0,0,0,1


In [28]:
# np.where 를 이용한 연속형 -> 이산형
# c1열의 평균값보다 크면 'high' , 작으면 'low' -> df['high_low'] 열 생성

df['high_low'] = np.where(df['C1']>df['C1'].mean(),'high','low')
df.head(3)

,C1,C2,high_low
0,1.095630,a,high
1,-0.309577,a,low
2,0.725752,a,high


In [29]:
# df 의 high_low 열로 그룹화 후 각 그룹 크기 출력
df.groupby(['high_low']).size()

high_low
high     8
low     12
dtype: int64

# 오후

In [30]:
# 데이터 재구조화 : pivot_table , stack , unstack , melt , wide_to_long

pivot : df.pivot(index,columns,values)  
pivot_table : df.pivot_table(data,index,columns,values)

In [31]:
data = pd.DataFrame({'cust_id': ['c1', 'c1', 'c1', 'c2', 'c2', 'c2', 'c3', 'c3', 'c3'],
   'prod_cd': ['p1', 'p2', 'p3', 'p1', 'p2', 'p3', 'p1', 'p2', 'p3'],
   'grade' : ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B'],
   'pch_amt': [30, 10, 0, 40, 15, 30, 0, 0, 10]})
data

,cust_id,prod_cd,grade,pch_amt
0,c1,p1,A,30
1,c1,p2,A,10
2,c1,p3,A,0
3,c2,p1,A,40
4,c2,p2,A,15
5,c2,p3,A,30
6,c3,p1,B,0
7,c3,p2,B,0
8,c3,p3,B,10


In [35]:
data_pivot = data.pivot(index='cust_id',columns='prod_cd',values='pch_amt')
data_pivot

prod_cd,p1,p2,p3
cust_id,,,
c1,30,10,0
c2,40,15,30
c3,0,0,10


In [32]:
data.pivot_table('pch_amt',index=['cust_id'],columns=['prod_cd'])

prod_cd,p1,p2,p3
cust_id,,,
c1,30,10,0
c2,40,15,30
c3,0,0,10


__pivot_table 은 되는데 pivot 함수는 지원하지 않는 기능__  
* 2 이상의 인덱스, 컬럼 종류  
* agg func 적용 X  ==> 'Index contains duplicate entries, cannot reshape'  
---
__agg function (집계함수) - 피봇팅 행,열을 설정했을 때 '값' 부분에 들어갈 데이터가 여러개일 경우 처리방법을 정해주어야함__  
ex) sum, mean, std 등등  

In [40]:
# index = cust_id , grade
# columns = prod_cd
# values = pch_amt

#data.pivot(index=['cust_id','grade'],columns='prod_cd',values='pch_amt')
data.pivot_table(index=['cust_id','grade'],columns='prod_cd',values='pch_amt')

,prod_cd,p1,p2,p3
cust_id,grade,,,
c1,A,30,10,0
c2,A,40,15,30
c3,B,0,0,10


In [41]:
data.pivot_table(index=['cust_id'],columns=['grade','prod_cd'],values=['pch_amt'])

pch_amt                            
grade         A                B           
prod_cd      p1    p2    p3   p1   p2    p3
cust_id                                    
c1         30.0  10.0   0.0  NaN  NaN   NaN
c2         40.0  15.0  30.0  NaN  NaN   NaN
c3          NaN   NaN   NaN  0.0  0.0  10.0

In [47]:
data.pivot_table(index='grade',columns='prod_cd',values='pch_amt',aggfunc=np.sum)

prod_cd,p1,p2,p3
grade,,,
A,70,25,30
B,0,0,10


In [48]:
# margin 옵션 행,열 둘다 적용하는? 옵션
data.pivot_table(index='grade',columns='prod_cd',values='pch_amt',aggfunc=np.sum,margins=True)

prod_cd,p1,p2,p3,All
grade,,,,
A,70,25,30,125
B,0,0,10,10
All,70,25,40,135


__stack , unstack__

In [49]:
mulindex = pd.MultiIndex.from_tuples([('c1','2020'),('c1','2021'),('c2','2020'),('c2','2021')])
mulindex

MultiIndex([('c1', '2020'),
            ('c1', '2021'),
            ('c2', '2020'),
            ('c2', '2021')],
           )

In [53]:
data = pd.DataFrame(np.arange(16).reshape(4,4),index=mulindex,columns=['p1','p2','p3','p4'])
data

p1  p2  p3  p4
c1 2020   0   1   2   3
   2021   4   5   6   7
c2 2020   8   9  10  11
   2021  12  13  14  15

In [55]:
# 데이터프레임 => stack => 시리즈

dataStacked = data.stack()
dataStacked

c1  2020  p1     0
          p2     1
          p3     2
          p4     3
    2021  p1     4
          p2     5
          p3     6
          p4     7
c2  2020  p1     8
          p2     9
          p3    10
          p4    11
    2021  p1    12
          p2    13
          p3    14
          p4    15
dtype: int32

In [56]:
dataStacked['c1','2020']

p1    0
p2    1
p3    2
p4    3
dtype: int32

In [65]:
dataStacked['c2']['2020']['p4']=np.nan
dataStacked

c1  2020  p1     0
          p2     1
          p3     2
          p4     3
    2021  p1     4
          p2     5
          p3     6
          p4     7
c2  2020  p1     8
          p2     9
          p3    10
          p4    11
    2021  p1    12
          p2    13
          p3    14
          p4    15
dtype: int32

In [66]:
dataStacked.unstack(level=-1)

p1  p2  p3  p4
c1 2020   0   1   2   3
   2021   4   5   6   7
c2 2020   8   9  10  11
   2021  12  13  14  15

In [67]:
dataStacked.unstack(level=0) # 리스트 인덱스처럼 가장 상위 분류부터 0으로 접근

c1  c2
2020 p1   0   8
     p2   1   9
     p3   2  10
     p4   3  11
2021 p1   4  12
     p2   5  13
     p3   6  14
     p4   7  15

In [68]:
df=dataStacked.unstack().reset_index()
df

,level_0,level_1,p1,p2,p3,p4
0,c1,2020,0,1,2,3
1,c1,2021,4,5,6,7
2,c2,2020,8,9,10,11
3,c2,2021,12,13,14,15


__melt :__

In [71]:
data = pd.DataFrame({'cust_ID' : ['C_001', 'C_001', 'C_002', 'C_002'],
   'prd_CD' : ['P_001', 'P_002', 'P_001', 'P_002'],
   'pch_cnt' : [1, 2, 3, 4],
   'pch_amt' : [100, 200, 300, 400]})
data

,cust_ID,prd_CD,pch_cnt,pch_amt
0,C_001,P_001,1,100
1,C_001,P_002,2,200
2,C_002,P_001,3,300
3,C_002,P_002,4,400


In [73]:
pd.melt(data,id_vars=['cust_ID','prd_CD'])

,cust_ID,prd_CD,variable,value
0,C_001,P_001,pch_cnt,1
1,C_001,P_002,pch_cnt,2
2,C_002,P_001,pch_cnt,3
3,C_002,P_002,pch_cnt,4
4,C_001,P_001,pch_amt,100
5,C_001,P_002,pch_amt,200
6,C_002,P_001,pch_amt,300
7,C_002,P_002,pch_amt,400


In [76]:
filepath='D:/downloads/padas_data/'
pew = pd.read_csv(filepath+'pew.csv')  # 종교와 연봉 사이 관계 데이터
pew.head(3)

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54


In [77]:
pew_long = pd.melt(pew,id_vars='religion',var_name='income',value_name='count')
pew_long.head(3)

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27


In [78]:
billboard = pd.read_csv(filepath+'billboard.csv')
billboard.head(3)

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
billboard_melt = pd.melt(billboard,id_vars=['year','artist','track','time','date.entered'],var_name='week',value_name='rating')
billboard_melt.head(10)

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0
5,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,wk1,51.0
6,2000,A*Teens,Dancing Queen,3:44,2000-07-08,wk1,97.0
7,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,wk1,84.0
8,2000,Aaliyah,Try Again,4:03,2000-03-18,wk1,59.0
9,2000,"Adams, Yolanda",Open My Heart,5:30,2000-08-26,wk1,76.0


In [84]:
country_timeseries = pd.read_csv(filepath+'country_timeseries.csv')
country_timeseries.head(1)

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN


In [85]:
ct_melt = pd.melt(country_timeseries,id_vars=['Date','Day'])
ct_melt.head(2)

,Date,Day,variable,value
0,1/5/2015,289,Cases_Guinea,2776.0
1,1/4/2015,288,Cases_Guinea,2775.0


In [88]:
ct_melt['status'] = ct_melt['variable'].apply(lambda x: x.split('_')[0])
ct_melt['country'] = ct_melt['variable'].apply(lambda x: x.split('_')[1])
ct_melt.head(3)

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea


In [89]:
weather = pd.read_csv(filepath+'weather.csv')
weather.head(1)

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN


In [90]:
weather_long = pd.melt(weather,id_vars=['id','year','month','element'],var_name='day',value_name='temp')
weather_long.head(3)

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN


In [93]:
weather_long_piv = weather_long.pivot_table(values='temp',index=['id','year','month','day'],columns='element')
weather_long_piv

element                 tmax  tmin
id      year month day            
MX17004 2010 1     d30  27.8  14.5
             2     d11  29.7  13.4
                   d2   27.3  14.4
                   d23  29.9  10.7
                   d3   24.1  14.4
             3     d10  34.5  16.8
                   d16  31.1  17.6
                   d5   32.1  14.2
             4     d27  36.3  16.7
             5     d27  33.2  18.2
             6     d17  28.0  17.5
                   d29  30.1  18.0
             7     d3   28.6  17.5
                   d14  29.9  16.5
             8     d23  26.4  15.0
                   d5   29.6  15.8
                   d29  28.0  15.3
                   d13  29.8  16.5
                   d25  29.7  15.6
                   d31  25.4  15.4
                   d8   29.0  17.3
             10    d5   27.0  14.0
                   d14  29.5  13.0
                   d15  28.7  10.5
                   d28  31.2  15.0
                   d7   28.1  12.9
             11    d2   31.3  16.3
                   d5   26.3   7.9
                   d27  27.7  14.2
                   d26  28.1  12.1
                   d4   27.2  12.0
             12    d1   29.9  13.8
                   d6   27.8  10.5